In [1]:
%%init_spark
launcher.master = "local[6]"
launcher.driver_memory = '8g'

In [2]:
import org.apache.spark.sql.functions._

Intitializing Scala interpreter ...

Spark Web UI available at http://pc:4040
SparkContext available as 'sc' (version = 3.1.2, master = local[6], app id = local-1631260398531)
SparkSession available as 'spark'


import org.apache.spark.sql.functions._


In [14]:
val rawDataPath = "/mnt/a/Projects/101DataSets/Open-Academic-Graph/raw-data/"
val dataPath = "/mnt/a/Projects/101DataSets/Open-Academic-Graph/data/"

rawDataPath: String = /mnt/a/Projects/101DataSets/Open-Academic-Graph/raw-data/
dataPath: String = /mnt/a/Projects/101DataSets/Open-Academic-Graph/data/


# Read Data

In [4]:
val authors = spark.read.parquet(rawDataPath+"spark/authors.parquet")

authors: org.apache.spark.sql.DataFrame = [id: string, n_citation: bigint ... 7 more fields]


In [5]:
authors.printSchema

root
 |-- id: string (nullable = true)
 |-- n_citation: long (nullable = true)
 |-- n_pubs: long (nullable = true)
 |-- name: string (nullable = true)
 |-- normalized_name: string (nullable = true)
 |-- org: string (nullable = true)
 |-- orgs: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- pubs: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- i: string (nullable = true)
 |    |    |-- r: long (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- t: string (nullable = true)



In [7]:
authors.show(2)

+----------+----------+------+------------------+------------------+----+----+-----------------+----+
|        id|n_citation|n_pubs|              name|   normalized_name| org|orgs|             pubs|tags|
+----------+----------+------+------------------+------------------+----+----+-----------------+----+
|2736393860|         0|     1|            马晓东|            马晓东|null|null|[{2738068945, 2}]|null|
|2736393861|         0|     1|Farras Amany Husna|farras amany husna|null|null|[{2738013386, 0}]|null|
+----------+----------+------+------------------+------------------+----+----+-----------------+----+
only showing top 2 rows



In [15]:
authors
.groupBy($"n_citation",$"n_pubs")
.count
.coalesce(1)
.write.option("header","true").mode("overwrite").csv(dataPath+"n_citation_n_pubs_count.csv")

In [16]:
authors.filter(!isnull($"tags"))
.select("pubs")
.show(false)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

|[{2039586356, 1}, {1498396823, 1}, {1505154190, 2}, {2808703988, 1}, {2613606427, 1}, {2085503384, 1}, {1520653788, 1}, {1523129306, 0}, {647125198, 0}, {2088436634, 2}, {1529914742, 0}, {1530377467, 1}, {1532039976, 3}, {1970364144, 2}, {2143088690, 0}, {2460372397, 2}, {2093460434, 1}, {2052401450, 0}, {1554638988, 1}, {1975287889, 2}, {2620389441, 1}, {1566184365, 2}, {2016387440, 0}, {2148071630, 1}, {1571592447, 0}, {193430011, 1}, {2097415527, 1}, {194625993, 1}, {2021339266, 0}, {2022606850, 1}, {2781482697, 2}, {2024132417, 0}, {1607801202, 2}, {2478382438, 2}, {2478895927, 0}, {2286178560, 0}, {1984609502, 0}, {2783606162, 1}, {2068561928, 0}, {2106170921, 0}, {2031565742, 2}, {2031969134, 0}, {1753261013, 0}, {1753504289, 1}, {1804802961, 1}, {2788990844, 2}, {2126114321, 0}, {2805781002, 4}]                                                                                                                                                                                          

|[{1972930906, 0}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

|[{1514441924, 0}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

...

|[{2136994117, 1}, {1964622094, 0}, {2086252758, 1}, {2767747960, 0}, {2047996726, 0}, {1019664267, 2}, {2888794556, 3}, {2053168885, 3}, {2055019542, 1}, {1571966732, 2}, {1978275577, 0}, {2059597595, 1}, {2061924559, 1}, {2152076888, 1}, {2062759102, 0}, {2154848766, 0}, {1986108401, 3}, {2896415191, 4}, {2068768383, 0}, {2303107758, 0}, {2514793257, 0}, {1997380212, 0}, {1840269408, 3}, {2078311314, 2}, {2738600695, 4}, {2176018869, 0}, {2124032736, 1}, {2753306693, 3}, {2128137392, 0}, {2797905475, 1}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

|[{2243842165, 0}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

|[{26000857, 0}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

|[{2079693459, 0}, {2321668714, 9}, {2524899724, 5}, {2041081871, 0}, {2415301460, 2}, {2415539309, 1}, {1964540280, 2}, {77301269, 4}, {2006815517, 0}, {2007153731, 2}, {2046782284, 1}, {1968932119, 0}, {2089244364, 2}, {2049735793, 0}, {1970524344, 6}, {1970844785, 2}, {2091886110, 2}, {2012900995, 1}, {1973504718, 1}, {1975218832, 1}, {2055413182, 16}, {2469968702, 1}, {2624961837, 1}, {2341649620, 0}, {1981353636, 1}, {2152402242, 0}, {2476089373, 1}, {2064674243, 5}, {2065052319, 9}, {2287759402, 4}, {2105484102, 20}, {2028991101, 2}, {2106176866, 0}, {2501903813, 1}, {2159632716, 13}, {2033401630, 0}, {2072696641, 1}, {2406688805, 1}, {2073795380, 3}, {2073795380, 3}, {1787027711, 12}, {2595251916, 27}, {2036555757, 0}, {2164418552, 2}, {1998774522, 0}, {2315758977, 2}, {1910619871, 2}]                                                                                                                                                                                                     

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [48]:
val authorPaper = authors.select($"id",$"n_citation",$"n_pubs",explode($"pubs").as("pc"))
.select($"id",$"n_citation",$"n_pubs",$"pc.i".as("paperId"),$"pc.r".as("order"))

authorPaper: org.apache.spark.sql.DataFrame = [id: string, n_citation: bigint ... 3 more fields]


In [49]:
authorPaper
.groupBy($"n_citation",$"n_pubs",$"order")
.count
.coalesce(1)
.write.option("header","true").mode("overwrite").csv(dataPath+"n_citation_n_pubs_order_count.csv")